# Лабораторная работа №1: Исследование алгоритма KNN (k-ближайших соседей)

Цель: построить бейзлайн-модели KNN для **классификации** и **регрессии** на заданных датасетах и оценить качество по выбранным метрикам.

Датасеты:
- **Классификация:** `Customer_support_data.csv` — классификация уровня удовлетворённости клиента.
- **Регрессия:** `cwurData.csv` — предсказание `world_rank` (мировой рейтинга университета).


## 1) Выбор метрик качества и обоснование

### Классификация
Будем использовать:
1. **Accuracy (доля верных ответов)** — простая и понятная метрика для общего качества, удобна для бейзлайна.
2. **F1-macro** — особенно важна при дисбалансе классов: усредняет F1 по классам **без** учёта их размера, поэтому показывает, насколько модель одинаково хорошо работает для всех классов.
Дополнительно (визуальный анализ):
- **Confusion Matrix** (матрица ошибок) — помогает понять, какие классы путаются между собой.

### Регрессия
Будем использовать:
1. **MAE (Mean Absolute Error)** — средняя абсолютная ошибка, интерпретируется в единицах целевой переменной (насколько в среднем ошибаемся).
2. **RMSE (Root Mean Squared Error)** — сильнее штрафует большие ошибки, полезно видеть “крупные промахи”.
3. **R² (коэффициент детерминации)** — показывает долю объяснённой дисперсии, удобен для сравнения моделей (чем ближе к 1, тем лучше).


In [174]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import (
    accuracy_score, f1_score, confusion_matrix, classification_report,
    mean_absolute_error, mean_squared_error, r2_score
)

# Базовые настройки вывода для удобства просмотра таблиц
pd.set_option("display.max_columns", 200)


In [175]:
def detect_target_column(df: pd.DataFrame, task: str) -> str:
    candidates = []
    if task == "classification":
        keywords = ["satisfaction", "satisfied", "satisf", "rating", "score", "label", "target", "class", "level"]
        for c in df.columns:
            cl = c.lower()
            if any(k in cl for k in keywords):
                candidates.append(c)
        # Если нашли кандидаты — берём первый; иначе берём последний столбец как запасной вариант
        return candidates[0] if candidates else df.columns[-1]
    else:
        # Для регрессии CWUR чаще всего целевая — world_rank
        if "world_rank" in df.columns:
            return "world_rank"
        if "World Rank" in df.columns:
            return "World Rank"
        return df.columns[-1]  # запасной вариант

# Функция построения препроцессинга для mixed-type данных
def make_preprocessor(X: pd.DataFrame) -> ColumnTransformer:
    num_cols = X.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = [c for c in X.columns if c not in num_cols]

    num_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    cat_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", num_pipe, num_cols),
            ("cat", cat_pipe, cat_cols),
        ],
        remainder="drop"
    )
    return preprocessor


In [176]:
# Загружаем датасет классификации
cls_path = "Customer_support_data.csv"
df_cls_full = pd.read_csv(cls_path)

# Берём случайные 20% данных для ускорения эксперимента и бейзлайна
df_cls = df_cls_full.sample(frac=0.2, random_state=42)

print("Full dataset shape:", df_cls_full.shape)
print("Sampled dataset shape (20%):", df_cls.shape)



Full dataset shape: (85907, 20)
Sampled dataset shape (20%): (17181, 20)


In [177]:
target_cls = detect_target_column(df_cls, task="classification")
X_cls = df_cls.drop(columns=[target_cls])
y_cls = df_cls[target_cls]

print("Target column (classification):", target_cls)
print("Shape X:", X_cls.shape, "| Shape y:", y_cls.shape)
print("y value counts (top):")
print(y_cls.value_counts().head(10))


Target column (classification): CSAT Score
Shape X: (17181, 19) | Shape y: (17181,)
y value counts (top):
CSAT Score
5    11934
4     2276
1     2241
3      503
2      227
Name: count, dtype: int64


In [178]:
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42, stratify=y_cls
)

# Делаем пайплайн: препроцессинг + KNN-классификатор
preprocessor_c = make_preprocessor(X_train_c)
knn_clf = KNeighborsClassifier(n_neighbors=5)

clf_pipeline = Pipeline(steps=[
    ("prep", preprocessor_c),
    ("model", knn_clf)
])


In [179]:
clf_pipeline.fit(X_train_c, y_train_c)  # Обучаем бейзлайн-модель KNN

y_pred_c = clf_pipeline.predict(X_test_c)  # Делаем предсказания на тесте

acc = accuracy_score(y_test_c, y_pred_c)
f1m = f1_score(y_test_c, y_pred_c, average="macro")

print(f"Accuracy: {acc:.4f}")
print(f"F1-macro: {f1m:.4f}")


Accuracy: 0.6270
F1-macro: 0.2132


In [180]:
cm = confusion_matrix(y_test_c, y_pred_c)  # Матрица ошибок помогает увидеть типичные путаницы классов
print("Confusion matrix:\n", cm)

print("\nClassification report:\n")
print(classification_report(y_test_c, y_pred_c))


Confusion matrix:
 [[  48    1    3   27  369]
 [   2    0    0    4   40]
 [   2    1    2   12   84]
 [  21    0    1   40  393]
 [ 143    0    8  171 2065]]

Classification report:

              precision    recall  f1-score   support

           1       0.22      0.11      0.14       448
           2       0.00      0.00      0.00        46
           3       0.14      0.02      0.03       101
           4       0.16      0.09      0.11       455
           5       0.70      0.87      0.77      2387

    accuracy                           0.63      3437
   macro avg       0.24      0.22      0.21      3437
weighted avg       0.54      0.63      0.57      3437



In [181]:
# Кросс-валидация для более устойчивой оценки качества (5 фолдов)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_validate(
    clf_pipeline, X_cls, y_cls,
    cv=cv,
    scoring={"accuracy": "accuracy", "f1_macro": "f1_macro"}
)

print("CV Accuracy:", np.round(cv_scores["test_accuracy"], 4), "| mean =", cv_scores["test_accuracy"].mean().round(4))
print("CV F1-macro:", np.round(cv_scores["test_f1_macro"], 4), "| mean =", cv_scores["test_f1_macro"].mean().round(4))


CV Accuracy: [0.6235 0.635  0.6266 0.6307 0.6278] | mean = 0.6287
CV F1-macro: [0.1987 0.2059 0.201  0.2079 0.2075] | mean = 0.2042


In [182]:
# Загрузим датасет регрессии (файл должен лежать рядом с ноутбуком или укажите путь)
reg_path = "cwurData.csv"
df_reg = pd.read_csv(reg_path)

df_reg.head()


,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [183]:
target_reg = detect_target_column(df_reg, task="regression")
X_reg = df_reg.drop(columns=[target_reg])
y_reg = df_reg[target_reg]

print("Target column (regression):", target_reg)
print("Shape X:", X_reg.shape, "| Shape y:", y_reg.shape)
print("y describe:")
print(pd.to_numeric(y_reg, errors="coerce").describe())


Target column (regression): world_rank
Shape X: (2200, 13) | Shape y: (2200,)
y describe:
count    2200.000000
mean      459.590909
std       304.320363
min         1.000000
25%       175.750000
50%       450.500000
75%       725.250000
max      1000.000000
Name: world_rank, dtype: float64


In [184]:
# На всякий случай приведём целевую к числу и уберём строки, где она не распарсилась
y_reg_num = pd.to_numeric(y_reg, errors="coerce")
mask = y_reg_num.notna()

X_reg = X_reg.loc[mask].copy()
y_reg_num = y_reg_num.loc[mask].copy()

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reg, y_reg_num, test_size=0.2, random_state=42
)


In [185]:
preprocessor_r = make_preprocessor(X_train_r)
knn_reg = KNeighborsRegressor(n_neighbors=5)

reg_pipeline = Pipeline(steps=[
    ("prep", preprocessor_r),
    ("model", knn_reg)
])


In [186]:
reg_pipeline.fit(X_train_r, y_train_r)  # Обучаем бейзлайн KNN-регрессор

y_pred_r = reg_pipeline.predict(X_test_r)  # Предсказываем на тестовой выборке

mae = mean_absolute_error(y_test_r, y_pred_r)
mse = mean_squared_error(y_test_r, y_pred_r)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_r, y_pred_r)

print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2:   {r2:.4f}")


MAE:  34.7432
RMSE: 54.6312
R2:   0.9674


In [187]:
# Кросс-валидация для регрессии (5 фолдов)
cv_r = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores_r = cross_validate(
    reg_pipeline, X_reg, y_reg_num,
    cv=cv_r,
    scoring={"mae": "neg_mean_absolute_error", "rmse": "neg_root_mean_squared_error", "r2": "r2"}
)

mae_cv = -cv_scores_r["test_mae"]
rmse_cv = -cv_scores_r["test_rmse"]
r2_cv = cv_scores_r["test_r2"]

print("CV MAE:", np.round(mae_cv, 4), "| mean =", mae_cv.mean().round(4))
print("CV RMSE:", np.round(rmse_cv, 4), "| mean =", rmse_cv.mean().round(4))
print("CV R2:", np.round(r2_cv, 4), "| mean =", r2_cv.mean().round(4))


CV MAE: [34.7432 36.6032 31.6664 33.0436 34.1845] | mean = 34.0482
CV RMSE: [54.6312 51.2494 45.7279 46.7943 49.1204] | mean = 49.5046
CV R2: [0.9674 0.9709 0.9769 0.9766 0.9745] | mean = 0.9733


## 2) Итоги по бейзлайну

Мы построили бейзлайны KNN:
- для классификации: оценили **Accuracy**, **F1-macro**, дополнительно посмотрели **матрицу ошибок** и отчёт по классам;
- для регрессии: оценили **MAE**, **RMSE**, **R²**.

Далее (в следующих пунктах/лабораторных) обычно делают:
- подбор `n_neighbors`, `weights`, `metric` (например, grid search);
- анализ влияния масштабирвания/кодирования признаков;
- сравнение с другими базовыми моделями.


## 3) Улучшение бейзлайна

### a) Гипотезы для улучшения качества

**Классификация (Customer_support_data):**
1. Подбор `n_neighbors`, `weights`, `metric` на кросс-валидации улучшит качество (Accuracy, F1-macro).
2. Масштабирование числовых признаков и One-Hot Encoding категориальных признаков критично для KNN (качество вырастет).
3. Использование взвешивания соседей (`weights='distance'`) поможет при “размытых” границах классов.

**Регрессия (cwurData):**
1. Подбор `n_neighbors`, `weights`, `metric` на кросс-валидации улучшит MAE/RMSE и повысит R².
2. Масштабирование числовых признаков важно для KNN-регрессии (уменьшит ошибки).
3. Удаление нерелевантных/шумных признаков и корректная обработка пропусков улучшит качество.


In [188]:
from sklearn.model_selection import GridSearchCV

# Функция: удобный вывод результатов поиска гиперпараметров
def show_best_grid(grid: GridSearchCV, title: str):
    print(title)
    print("Best params:", grid.best_params_)  # Печатаем лучшие параметры по CV
    print("Best CV score:", round(grid.best_score_, 5))


### b) Проверка гипотез (классификация): подбор гиперпараметров KNN

Подбираем параметры:
- `model__n_neighbors`: число соседей
- `model__weights`: равные веса или веса по расстоянию
- `model__metric`: метрика расстояния (евклидова / манхэттенская)

Оптимизируем по **F1-macro**, т.к. она устойчива к дисбалансу классов.


In [189]:
# Пайплайн уже есть: clf_pipeline = Pipeline([("prep", ...), ("model", KNeighborsClassifier(...))])
# Здесь делаем грид по параметрам KNN для улучшения качества
param_grid_cls = {
    "model__n_neighbors": [3, 5, 7, 11, 15, 21],
    "model__weights": ["uniform", "distance"],
    "model__metric": ["minkowski", "manhattan"]
}

cv_cls = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_cls = GridSearchCV(
    estimator=clf_pipeline,
    param_grid=param_grid_cls,
    scoring="f1_macro",
    cv=cv_cls,
    n_jobs=-1
)


In [190]:
grid_cls.fit(X_train_c, y_train_c)  # Подбираем параметры на train по CV

show_best_grid(grid_cls, "KNN Classifier GridSearch (optimize F1-macro):")  # Печатаем итог подбора


KNN Classifier GridSearch (optimize F1-macro):
Best params: {'model__metric': 'manhattan', 'model__n_neighbors': 3, 'model__weights': 'distance'}
Best CV score: 0.21187


### c-d-e) Улучшенный бейзлайн (классификация) + обучение + оценка на тесте

Берём лучшую модель из GridSearchCV и оцениваем её на тестовой выборке по метрикам:
- Accuracy
- F1-macro
- Confusion Matrix


In [191]:
best_clf = grid_cls.best_estimator_  # Берём лучший пайплайн из поиска

y_pred_c_best = best_clf.predict(X_test_c)  # Предсказания улучшенной модели на тесте

acc_best = accuracy_score(y_test_c, y_pred_c_best)
f1m_best = f1_score(y_test_c, y_pred_c_best, average="macro")

print(f"Improved Accuracy: {acc_best:.4f}")
print(f"Improved F1-macro: {f1m_best:.4f}")


Improved Accuracy: 0.5965
Improved F1-macro: 0.2111


In [192]:
cm_best = confusion_matrix(y_test_c, y_pred_c_best)  # Матрица ошибок для улучшенной модели
print("Improved confusion matrix:\n", cm_best)

print("\nImproved classification report:\n")
print(classification_report(y_test_c, y_pred_c_best))


Improved confusion matrix:
 [[  71    2    9   26  340]
 [   3    0    0   11   32]
 [   7    3    2    9   80]
 [  35    3   10   31  376]
 [ 167   17   52  205 1946]]

Improved classification report:

              precision    recall  f1-score   support

           1       0.25      0.16      0.19       448
           2       0.00      0.00      0.00        46
           3       0.03      0.02      0.02       101
           4       0.11      0.07      0.08       455
           5       0.70      0.82      0.75      2387

    accuracy                           0.60      3437
   macro avg       0.22      0.21      0.21      3437
weighted avg       0.54      0.60      0.56      3437



### b) Проверка гипотез (регрессия): подбор гиперпараметров KNN

Подбираем параметры:
- `model__n_neighbors`
- `model__weights`
- `model__metric`

Оптимизируем по **MAE** (через `neg_mean_absolute_error`), т.к. MAE проще интерпретировать.


In [193]:
param_grid_reg = {
    "model__n_neighbors": [3, 5, 7, 11, 15, 21, 31],
    "model__weights": ["uniform", "distance"],
    "model__metric": ["minkowski", "manhattan"]
}

cv_reg = KFold(n_splits=5, shuffle=True, random_state=42)

grid_reg = GridSearchCV(
    estimator=reg_pipeline,
    param_grid=param_grid_reg,
    scoring="neg_mean_absolute_error",
    cv=cv_reg,
    n_jobs=-1
)


In [194]:
grid_reg.fit(X_train_r, y_train_r)  # Подбираем параметры на train по CV

show_best_grid(grid_reg, "KNN Regressor GridSearch (optimize MAE):")  # Best score будет отрицательным (так устроен sklearn)


KNN Regressor GridSearch (optimize MAE):
Best params: {'model__metric': 'manhattan', 'model__n_neighbors': 7, 'model__weights': 'distance'}
Best CV score: -30.4542


### c-d-e) Улучшенный бейзлайн (регрессия) + обучение + оценка на тесте

Берём лучшую модель из GridSearchCV и оцениваем на тесте:
- MAE
- RMSE
- R²


In [195]:
best_reg = grid_reg.best_estimator_  # Лучший пайплайн регрессии

y_pred_r_best = best_reg.predict(X_test_r)  # Предсказания улучшенной модели

mae_best = mean_absolute_error(y_test_r, y_pred_r_best)
rmse_best = mean_squared_error(y_test_r, y_pred_r_best) ** 0.5
r2_best = r2_score(y_test_r, y_pred_r_best)

print(f"Improved MAE:  {mae_best:.4f}")
print(f"Improved RMSE: {rmse_best:.4f}")
print(f"Improved R2:   {r2_best:.4f}")


Improved MAE:  30.5517
Improved RMSE: 53.1147
Improved R2:   0.9692


### f) Сравнение результатов с бейзлайном (пункт 2)

Сравним метрики «baseline» vs «improved»:
- Классификация: Accuracy, F1-macro
- Регрессия: MAE, RMSE, R²


In [196]:
# В этом блоке предполагается, что baseline-метрики уже посчитаны ранее:
# acc, f1m (классификация) и mae, rmse, r2 (регрессия)

comparison = pd.DataFrame({
    "Task": ["Classification", "Classification", "Regression", "Regression", "Regression"],
    "Metric": ["Accuracy", "F1-macro", "MAE", "RMSE", "R2"],
    "Baseline": [acc, f1m, mae, rmse, r2],
    "Improved": [acc_best, f1m_best, mae_best, rmse_best, r2_best]
})

comparison["Delta (Improved - Baseline)"] = comparison["Improved"] - comparison["Baseline"]  # Разница качества
comparison


,Task,Metric,Baseline,Improved,Delta (Improved - Baseline)
0,Classification,Accuracy,0.627000,0.596450,-0.030550
1,Classification,F1-macro,0.213179,0.211097,-0.002082
2,Regression,MAE,34.743182,30.551661,-4.191521
3,Regression,RMSE,54.631164,53.114691,-1.516473
4,Regression,R2,0.967445,0.969227,0.001782


### g) Выводы

1. Подбор гиперпараметров алгоритма KNN с использованием кросс-валидации **не привёл к улучшению качества моделей** по сравнению с бейзлайном.

2. Для задачи классификации было зафиксировано **небольшое снижение метрик Accuracy и F1-macro**, что говорит о том, что исходные параметры KNN (`n_neighbors=5`, `weights='uniform'`) уже были близки к оптимальным для выбранного поднабора данных (20 % датасета).

3. Для задачи регрессии подбор гиперпараметров позволил **снизить абсолютную и квадратичную ошибку (MAE, RMSE)**, однако прирост оказался несущественным, а коэффициент детерминации R² изменился незначительно.

4. Полученные результаты показывают, что:
   - алгоритм KNN чувствителен к структуре данных;
   - для выбранных датасетов и объёма данных потенциал улучшения за счёт подбора гиперпараметров ограничен.

5. Таким образом, в рамках данной лабораторной работы **базовая модель KNN может рассматриваться как адекватный бейзлайн**, а дальнейшее улучшение качества целесообразно искать в:
   - формировании новых признаков;
   - уменьшении размерности признакового пространства;
   - использовании других моделей машинного обучения.


## 4) Имплементация алгоритма машинного обучения (KNN)

В этом разделе реализуем алгоритм **k-ближайших соседей (KNN)** самостоятельно:
- KNN для **классификации** (majority vote, опционально weights='distance')
- KNN для **регрессии** (среднее по соседям, опционально weights='distance')

Далее:
- обучим имплементированные модели на тех же train/test,
- оценим качество по тем же метрикам,
- сравним с результатами из пункта 2 (baseline) и пункта 3 (improved),
- добавим техники улучшенного бейзлайна (препроцессинг: imputer + scaler + OHE) и повторим сравнение.


In [197]:
import numpy as np

# Небольшая функция для RMSE, чтобы не повторять код
def rmse_fn(y_true, y_pred):
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))  # RMSE удобнее интерпретировать в единицах y


In [198]:
class KNNBaseBatched:
    def __init__(self, n_neighbors=5, weights="uniform", p=2, batch_size=128):
        self.n_neighbors = int(n_neighbors)
        self.weights = weights
        self.p = p
        self.batch_size = int(batch_size)  # Батчи ограничивают потребление памяти

    def fit(self, X, y):
        self.X_train_ = np.asarray(X, dtype=np.float32)
        self.y_train_ = np.asarray(y)
        return self  # Храним train, но считаем расстояния порциями

    def _topk_for_batch(self, Xb):
        Xb = np.asarray(Xb, dtype=np.float32)
        preds = []
        for i in range(Xb.shape[0]):
            diff = np.abs(self.X_train_ - Xb[i])  # Считаем расстояния только для одного объекта
            dist = np.sum(diff ** self.p, axis=1) ** (1 / self.p)
            idx = np.argpartition(dist, self.n_neighbors - 1)[: self.n_neighbors]
            preds.append((idx, dist[idx]))  # Храним только k соседей, а не все расстояния
        return preds


In [199]:
class KNNClassifierScratchBatched(KNNBaseBatched):
    def predict(self, X):
        X = np.asarray(X, dtype=np.float32)
        out = []
        for s in range(0, X.shape[0], self.batch_size):
            batch = X[s:s+self.batch_size]
            topk = self._topk_for_batch(batch)
            for idx, d in topk:
                neigh_y = self.y_train_[idx]
                w = 1.0 / (d + 1e-12) if self.weights == "distance" else np.ones_like(d)
                labels, inv = np.unique(neigh_y, return_inverse=True)
                scores = np.bincount(inv, weights=w, minlength=len(labels))
                out.append(labels[np.argmax(scores)])  # Majority vote (или weighted vote)
        return np.array(out)


In [200]:
class KNNRegressorScratchBatched(KNNBaseBatched):
    def predict(self, X):
        X = np.asarray(X, dtype=np.float32)
        out = []
        for s in range(0, X.shape[0], self.batch_size):
            batch = X[s:s+self.batch_size]
            topk = self._topk_for_batch(batch)
            for idx, d in topk:
                neigh_y = self.y_train_[idx].astype(np.float32)
                if self.weights == "distance":
                    w = 1.0 / (d + 1e-12)  # Взвешиваем вклад соседей по расстоянию
                    out.append(float(np.sum(neigh_y * w) / np.sum(w)))
                else:
                    out.append(float(np.mean(neigh_y)))  # Среднее по соседям
        return np.array(out, dtype=np.float32)


### Подготовка данных для scratch-версии (только числовые признаки)

Самописный KNN работает с числовыми матрицами, поэтому:
- берём только numeric-признаки,
- заполняем пропуски медианой,
- стандартизируем признаки (важно для расстояний).


In [201]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Препроцессинг для scratch: импаутинг + масштабирование только числовых признаков
num_imputer = SimpleImputer(strategy="median")
num_scaler = StandardScaler()  # Масштабирование важно, иначе признаки с большим масштабом доминируют


In [202]:
# --- Классификация: scratch данные (только numeric) ---
Xc_train_num = X_train_c.select_dtypes(include=["number"])
Xc_test_num = X_test_c.select_dtypes(include=["number"])

Xc_train_num_imp = num_imputer.fit_transform(Xc_train_num)
Xc_test_num_imp = num_imputer.transform(Xc_test_num)

Xc_train_num_scaled = num_scaler.fit_transform(Xc_train_num_imp)
Xc_test_num_scaled = num_scaler.transform(Xc_test_num_imp)

print("Numeric features (classification):", Xc_train_num.shape[1])  # Проверяем число числовых признаков


Numeric features (classification): 2


In [203]:
# --- Регрессия: scratch данные (только numeric) ---
Xr_train_num = X_train_r.select_dtypes(include=["number"])
Xr_test_num = X_test_r.select_dtypes(include=["number"])

Xr_train_num_imp = num_imputer.fit_transform(Xr_train_num)
Xr_test_num_imp = num_imputer.transform(Xr_test_num)

Xr_train_num_scaled = num_scaler.fit_transform(Xr_train_num_imp)
Xr_test_num_scaled = num_scaler.transform(Xr_test_num_imp)

print("Numeric features (regression):", Xr_train_num.shape[1])  # Проверяем число числовых признаков


Numeric features (regression): 11


### 4b–4c) Обучение и оценка scratch-KNN (классификация)

Сравниваем по тем же метрикам: Accuracy и F1-macro.


In [204]:
knn_cls_s = KNNClassifierScratch(n_neighbors=5, weights="uniform", p=2)
knn_cls_s.fit(Xc_train_num_scaled, y_train_c)  # Обучаем самописный KNN на числовых признаках

y_pred_cls_s = knn_cls_s.predict(Xc_test_num_scaled)  # Предсказываем на тесте

acc_s = accuracy_score(y_test_c, y_pred_cls_s)
f1m_s = f1_score(y_test_c, y_pred_cls_s, average="macro")
print(f"Scratch KNN (cls) Accuracy: {acc_s:.4f}, F1-macro: {f1m_s:.4f}")


Scratch KNN (cls) Accuracy: 0.6730, F1-macro: 0.1856


### 4b–4c) Обучение и оценка scratch-KNN (регрессия)

Сравниваем по MAE, RMSE, R².


In [205]:
knn_reg_s = KNNRegressorScratch(n_neighbors=5, weights="uniform", p=2)
knn_reg_s.fit(Xr_train_num_scaled, y_train_r)  # Обучаем самописный KNN-регрессор

y_pred_reg_s = knn_reg_s.predict(Xr_test_num_scaled)  # Предсказываем на тесте

mae_s = mean_absolute_error(y_test_r, y_pred_reg_s)
rmse_s = rmse_fn(y_test_r.to_numpy(), y_pred_reg_s)
r2_s = r2_score(y_test_r, y_pred_reg_s)
print(f"Scratch KNN (reg) MAE: {mae_s:.4f}, RMSE: {rmse_s:.4f}, R2: {r2_s:.4f}")


Scratch KNN (reg) MAE: 33.4868, RMSE: 55.2288, R2: 0.9667


### 4d) Сравнение scratch-моделей с бейзлайном из пункта 2

Сравним:
- классификация: Accuracy, F1-macro
- регрессия: MAE, RMSE, R²


In [206]:
# Предполагается, что baseline-метрики из пункта 2 уже есть: acc, f1m, mae, rmse, r2
comp_p2 = pd.DataFrame({
    "Task": ["Classification", "Classification", "Regression", "Regression", "Regression"],
    "Metric": ["Accuracy", "F1-macro", "MAE", "RMSE", "R2"],
    "Baseline (p2)": [acc, f1m, mae, rmse, r2],
    "Scratch": [acc_s, f1m_s, mae_s, rmse_s, r2_s]
})

comp_p2["Delta (Scratch - Baseline)"] = comp_p2["Scratch"] - comp_p2["Baseline (p2)"]  # Разница относительно бейзлайна
comp_p2


,Task,Metric,Baseline (p2),Scratch,Delta (Scratch - Baseline)
0,Classification,Accuracy,0.627000,0.672971,0.045970
1,Classification,F1-macro,0.213179,0.185558,-0.027621
2,Regression,MAE,34.743182,33.486818,-1.256364
3,Regression,RMSE,54.631164,55.228821,0.597657
4,Regression,R2,0.967445,0.966729,-0.000716


### 4e) Выводы по scratch-версии

- Самописный KNN воспроизводит идею алгоритма k-ближайших соседей (расстояния → соседи → агрегация ответов).
- Различия с sklearn могут возникать из-за:
  - использования только числовых признаков (без категориальных),
  - отличий в обработке данных и оптимизациях sklearn.


## 4f–4h) Scratch + техники улучшенного бейзлайна (без MemoryError)

Важно: после OneHotEncoder матрица обычно sparse.  
Чтобы самописный KNN работал, возьмём **малый поднабор** train/test после препроцессинга,
иначе расчёты будут слишком долгими (даже без MemoryError).


In [207]:
# Полный препроцессинг (числовые + категориальные признаки) как в улучшенном бейзлайне
prep_c_full = make_preprocessor(X_train_c)

Xc_train_full = prep_c_full.fit_transform(X_train_c)
Xc_test_full = prep_c_full.transform(X_test_c)

print("Prepared shapes (classification):", Xc_train_full.shape, Xc_test_full.shape)


Prepared shapes (classification): (13744, 54821) (3437, 54821)


In [208]:
# Полный препроцессинг для регрессии (как в пункте 3с)
prep_r_full = make_preprocessor(X_train_r)

Xr_train_full = prep_r_full.fit_transform(X_train_r)
Xr_test_full = prep_r_full.transform(X_test_r)

print("Prepared shapes (regression):", Xr_train_full.shape, Xr_test_full.shape)


Prepared shapes (regression): (1760, 1047) (440, 1047)


In [209]:
# Берём небольшой поднабор, чтобы самописный KNN отработал быстро (можно увеличить при желании)
rng = np.random.RandomState(42)
train_limit = min(2000, Xc_train_full.shape[0])
test_limit = min(600, Xc_test_full.shape[0])

train_idx = rng.choice(Xc_train_full.shape[0], size=train_limit, replace=False)
test_idx = rng.choice(Xc_test_full.shape[0], size=test_limit, replace=False)

Xc_train_sub = Xc_train_full[train_idx]
Xc_test_sub = Xc_test_full[test_idx]
y_train_sub = y_train_c.iloc[train_idx]
y_test_sub = y_test_c.iloc[test_idx]

print("Subsample shapes (cls):", Xc_train_sub.shape, Xc_test_sub.shape)  # Контроль размеров поднабора


Subsample shapes (cls): (2000, 54821) (600, 54821)


In [210]:
# Конвертируем sparse->dense только для поднабора (это уже не убьёт память)
Xc_train_sub_dense = Xc_train_sub.toarray() if hasattr(Xc_train_sub, "toarray") else Xc_train_sub
Xc_test_sub_dense = Xc_test_sub.toarray() if hasattr(Xc_test_sub, "toarray") else Xc_test_sub

knn_cls_s2 = KNNClassifierScratchBatched(
    n_neighbors=grid_cls.best_params_["model__n_neighbors"],
    weights=grid_cls.best_params_["model__weights"],
    p=2,
    batch_size=64
)
knn_cls_s2.fit(Xc_train_sub_dense, y_train_sub)  # Обучаем самописный KNN на препроцессенных данных (поднабор)

y_pred_cls_s2 = knn_cls_s2.predict(Xc_test_sub_dense)  # Предсказываем без выделения терабайт памяти

acc_s2 = accuracy_score(y_test_sub, y_pred_cls_s2)
f1m_s2 = f1_score(y_test_sub, y_pred_cls_s2, average="macro")
print(f"Scratch+Prep (cls, subsample) Accuracy: {acc_s2:.4f}, F1-macro: {f1m_s2:.4f}")


Scratch+Prep (cls, subsample) Accuracy: 0.6233, F1-macro: 0.2319


In [211]:
# Аналогично для регрессии: берём поднабор после препроцессинга
rng = np.random.RandomState(42)
train_limit_r = min(2500, Xr_train_full.shape[0])
test_limit_r = min(800, Xr_test_full.shape[0])

train_idx_r = rng.choice(Xr_train_full.shape[0], size=train_limit_r, replace=False)
test_idx_r = rng.choice(Xr_test_full.shape[0], size=test_limit_r, replace=False)

Xr_train_sub = Xr_train_full[train_idx_r]
Xr_test_sub = Xr_test_full[test_idx_r]
y_train_sub_r = y_train_r.iloc[train_idx_r]
y_test_sub_r = y_test_r.iloc[test_idx_r]

Xr_train_sub_dense = Xr_train_sub.toarray() if hasattr(Xr_train_sub, "toarray") else Xr_train_sub
Xr_test_sub_dense = Xr_test_sub.toarray() if hasattr(Xr_test_sub, "toarray") else Xr_test_sub

knn_reg_s2 = KNNRegressorScratchBatched(
    n_neighbors=grid_reg.best_params_["model__n_neighbors"],
    weights=grid_reg.best_params_["model__weights"],
    p=2,
    batch_size=64
)
knn_reg_s2.fit(Xr_train_sub_dense, y_train_sub_r)  # Обучаем на поднаборе, чтобы расчёты были быстрыми

y_pred_reg_s2 = knn_reg_s2.predict(Xr_test_sub_dense)

mae_s2 = mean_absolute_error(y_test_sub_r, y_pred_reg_s2)
rmse_s2 = rmse_fn(y_test_sub_r.to_numpy(), y_pred_reg_s2)
r2_s2 = r2_score(y_test_sub_r, y_pred_reg_s2)

print(f"Scratch+Prep (reg, subsample) MAE: {mae_s2:.4f}, RMSE: {rmse_s2:.4f}, R2: {r2_s2:.4f}")


Scratch+Prep (reg, subsample) MAE: 34.1384, RMSE: 54.9068, R2: 0.9671


## 4i) Сравнение с пунктом 3 (улучшенный бейзлайн)

Важный момент: scratch+prep считался на **поднаборах**, поэтому сравнение с пунктом 3 корректно
лишь как приближённое (по скорости/идеологии), а не как полностью честное по одинаковым данным.


In [212]:
comp_p3 = pd.DataFrame({
    "Task": ["Classification", "Classification", "Regression", "Regression", "Regression"],
    "Metric": ["Accuracy", "F1-macro", "MAE", "RMSE", "R2"],
    "Improved (p3)": [acc_best, f1m_best, mae_best, rmse_best, r2_best],
    "Scratch+Prep (subsample)": [acc_s2, f1m_s2, mae_s2, rmse_s2, r2_s2]
})

comp_p3["Delta"] = comp_p3["Scratch+Prep (subsample)"] - comp_p3["Improved (p3)"]  # Разница с improved
comp_p3


,Task,Metric,Improved (p3),Scratch+Prep (subsample),Delta
0,Classification,Accuracy,0.596450,0.623333,0.026883
1,Classification,F1-macro,0.211097,0.231928,0.020831
2,Regression,MAE,30.551661,34.138374,3.586713
3,Regression,RMSE,53.114691,54.906828,1.792136
4,Regression,R2,0.969227,0.967116,-0.002112


## Выводы по сравнению моделей (пункт 4i–4j)

На основе полученных метрик можно сделать следующие выводы:

### Классификация
1. Самописная модель KNN с полным препроцессингом (**Scratch + Prep**) показала
   **улучшение качества** по сравнению с улучшенным бейзлайном (пункт 3):
   - Accuracy увеличилась с 0.596 до 0.623;
   - F1-macro выросла с 0.211 до 0.232.
2. Рост F1-macro особенно важен, так как эта метрика отражает более равномерное
   качество классификации по всем классам и менее чувствительна к дисбалансу данных.
3. Полученные улучшения свидетельствуют о том, что самописная реализация KNN
   корректно использует информацию о расстояниях между объектами после препроцессинга.

### Регрессия
1. Для задачи регрессии модель **Scratch + Prep** показала ухудшение качества:
   - MAE и RMSE увеличились, что означает рост средней и квадратичной ошибок прогноза;
   - коэффициент детерминации R² немного снизился.
2. Вероятной причиной ухудшения является использование **поднабора данных** для
   самописной модели, а также отсутствие оптимизаций, присутствующих в реализации sklearn.
3. Несмотря на ухудшение метрик, значения ошибок остаются сопоставимыми с улучшенным
   бейзлайном, что подтверждает корректность реализованного алгоритма.

### Общий вывод
1. Самописная реализация KNN позволяет достичь качества, сопоставимого с библиотечной,
   особенно в задаче классификации.
2. Для задач регрессии библиотечная реализация sklearn оказывается более устойчивой
   и предпочтительной для практического использования.
3. Реализация алгоритма «с нуля» полезна прежде всего для понимания принципов работы
   KNN, тогда как для реальных задач целесообразно использовать оптимизированные
   реализации из библиотек машинного обучения.
